In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://psmag.com/.image/t_share/MTUwMjg0NDIxMTE3NzgyMDEy/college-2.jpg)

# K-Means Clustering Project

This data has 777 observations on the following 18 variables.

1. Private A factor with levels No and Yes indicating private or public university
2. Apps Number of applications received
3. Accept Number of applications accepted
4. Enroll Number of new students enrolled
5. Top10perc Pct. new students from top 10% of H.S. class
6. Top25perc Pct. new students from top 25% of H.S. class
7. F.Undergrad Number of fulltime undergraduates
8. P.Undergrad Number of parttime undergraduates
9. Outstate Out-of-state tuition
10. Room.Board Room and board costs
11. Books Estimated book costs
12. Personal Estimated personal spending
13. PhD Pct. of faculty with Ph.D.’s
14. Terminal Pct. of faculty with terminal degree
15. S.F.Ratio Student/faculty ratio
16. perc.alumni Pct. alumni who donate
17. Expend Instructional expenditure per student
18. Grad.Rate Graduation rate

In [ ]:
df = pd.read_csv("../input/college-data/data.csv")

In [ ]:
df.head()

In [ ]:
df.info() #No missing values are in the dataset

In [ ]:
df.describe().T

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# EDA

In [ ]:
sns.countplot(data=df, x='private')

In [ ]:
plt.figure(figsize=(12,8))
plt.title('Enrollment Statistics in Private vs Public Universities')
sns.histplot(data=df, x="enroll", hue="private", multiple="dodge", shrink=.9)

In [ ]:
plt.figure(figsize=(12,8))
plt.title('Alumni Statistics in Private vs Public Universities')
sns.histplot(data=df, x="perc_alumni", hue="private", multiple="stack")

In [ ]:
sns.set_style('darkgrid')
sns.lmplot('room_board','grad_rate',data=df, hue='private',
           palette='coolwarm',size=6,aspect=1,fit_reg=False)

In [ ]:
sns.set_style('darkgrid')
sns.color_palette("magma", as_cmap=True)
g = sns.FacetGrid(df,hue="private",size=6,aspect=2)
g = g.map(plt.hist,'outstate',bins=20,alpha=0.7)

In [ ]:
sns.histplot(data=df, x="grad_rate")

In [ ]:
sns.set_style('darkgrid')
g = sns.FacetGrid(df,hue="private",palette='coolwarm',size=6,aspect=2)
g = g.map(plt.hist,'grad_rate',bins=20,alpha=0.7)

In [ ]:
df['encoded'] = df['private'].map({'Yes':1,'No':0})

In [ ]:
df.head()

In [ ]:
features_mean =list(df.columns[1:18])
# split dataframe into two based on diagnosis
df_Private=df[df['encoded'] ==1]
df_Public =df[df['encoded'] ==0]

In [ ]:
plt.rcParams.update({'font.size': 10})
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(13,10))
axes = axes.ravel()
for idx,ax in enumerate(axes):
    ax.figure
    binwidth= (max(df[features_mean[idx]]) - min(df[features_mean[idx]]))/50
    ax.hist([df_Private[features_mean[idx]],df_Public[features_mean[idx]]], bins=np.arange(min(df[features_mean[idx]]), max(df[features_mean[idx]]) + binwidth, binwidth) , alpha=0.5,stacked=True,  label=['Private','Public'],color=['b','y'])
    ax.legend(loc='upper right')
    ax.set_title(features_mean[idx])
plt.tight_layout()
plt.show()

# K Means Cluster 

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
kmeans.fit(df.drop(['private', 'encoded'], axis=1))

In [ ]:
kmeans.cluster_centers_

In [ ]:
df['encoded'] = df['private'].map({'Yes':1,'No':0})

# Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(df['encoded'],kmeans.labels_))
print(classification_report(df['encoded'],kmeans.labels_))